In [2]:
from langchain.agents import create_sql_agent,create_openai_functions_agent,create_openapi_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
import os
from langchain_community.llms import AzureOpenAI
from langchain.agents.agent_types import AgentType
os.environ["OPENAI_API_TYPE"] = "azure"
import openai



In [3]:
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242"
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"

In [4]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["OPENAI_API_BASE"] = "https://like-card-test.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "85889c7998dd4adb9a4c89abe56b1242"
os.environ["OPENAI_API_ENDPOINT"]="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"

In [5]:
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242" 
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo-instruct/chat/completions?api-version=2023-07-01-preview"

In [6]:
f = open("Credintials.txt", "r")
email = None
password = None
for i, line in enumerate(f): 
    if i == 0:
        email = line.strip()  # Assign the first line to email variable
    elif i == 1:
        password = line.strip()  # Assign the second line to password variable

f.close()

In [7]:
server = 'anmx3c2yurjethctgyba6xtuaq-vkvbn7t4322edeguc3ufhjtjeu.datawarehouse.pbidedicated.windows.net'
database = 'likeCard'
driver='ODBC Driver 18 for SQL Server'
user= email # use %40 inplace of @
pass1=password # use %40 inplace of @   
db_string =f'mssql+pyodbc://{user}:{pass1}@{server}/{database}?driver={driver}&Trusted_Connection=no&Authentication=ActiveDirectoryInteractive'
dp=SQLDatabase.from_uri(db_string)


In [8]:
llm = AzureOpenAI( model_name="gpt-35-turbo-instruct",deployment_name="gpt-35-turbo-instruct",temperature=0.2)

c:\Users\AbdelKaderTalaat\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `AzureOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureOpenAI instead.
  warn_deprecated(


In [9]:
toolkit = SQLDatabaseToolkit(db=dp,llm=llm)


In [10]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True ,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,

)

In [11]:
from langchain.memory import ConversationBufferMemory


In [134]:
SQL_SUFFIX = """Begin!
history: {history}
Task: **Using the available SQL Server database, generate and execute a concise SQL query to accurately answer the following question:**

{query}

**Key Considerations:**

- **Identify relevant tables:** Determine the tables containing the necessary information to answer the question.
- **Join tables appropriately:** Establish relationships between tables using appropriate joins (e.g., INNER JOIN, LEFT JOIN, etc.) if required.
- **Apply aggregations:** Use functions like SUM(), COUNT(), AVG(), MAX(), MIN(), and GROUP BY to perform calculations or group data if needed.
- **Select relevant columns:** Retrieve only the specific columns essential for providing the desired insights.
- **Structure the output:** Return the query results in a clear and informative format, such as a list of rows or a formatted table.
- **if the out contain numbers seprate the number with comma for example 154123456 to be 154,123,456

**Additional Context (optional):**

- If relevant, provide brief descriptions of key tables and columns to aid understanding.
- Clarify any specific business rules or constraints that should be considered.

**Agent Scratchpad:**

{agent_scratchpad}
"""


In [135]:
agent_executor = create_sql_agent(   
    llm,
    SQLDatabaseToolkit(db=dp, llm=llm),
    agent_executor_kwargs={
        "memory": ConversationBufferMemory(
            input_key="query", memory_key="history", return_messages= True

        )
    },
    
    suffix=SQL_SUFFIX,
    input_variables=["query", "history", "agent_scratchpad"],
    verbose= True
)  

In [144]:
query = "total sales 2023 monthly"

In [147]:
output=agent_executor.invoke(query)



> Entering new AgentExecutor chain...
Thought: Identify relevant tables
Action: sql_db_list_tables
Action Input: ''
Observation: BL_Mapping_Table, Boost_customers, Customers_DAX_table, DAX_table, GMV_Calender_DAX_table, GMV_DimDate_DAX_table, Same_period, dimDate, dimDateEX, dim_B2B_Sales_Channel, dim_Target, dim_allusers, dim_balance, dim_categories, dim_industry, dim_log_status, dim_parent_categories, dim_payment_methods, dim_pos_sales_channel, dim_products, dim_segments, dim_stores, dim_users_b2b, dim_users_b2c, dim_users_pos, exploration, fact_b2b_trans, fact_b2c_trans, fact_points, fact_pos_trans, fact_salla_trans, fact_transactions, fact_vouchers
Thought:Action: sql_db_schema
Action Input: Boost_customers, Customers_DAX_table, DAX_table, GMV_Calender_DAX_table, GMV_DimDate_DAX_table, Same_period, dimDate, dimDateEX, dim_B2B_Sales_Channel, dim_Target, dim_allusers, dim_balance, dim_categories, dim_industry, dim_log_status, dim_parent_categories, dim_payment_methods, dim_pos_sale

OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: An existing connection was forcibly closed by the remote host.\r\n (10054) (SQLExecDirectW); [08S01] [Microsoft][ODBC Driver 18 for SQL Server]Communication link failure (10054)')
[SQL: SELECT TOP 3 [Boost_customers].user_id, [Boost_customers].[AOV], [Boost_customers].[Frequency], [Boost_customers].[OBJ], [Boost_customers].[AGP], [Boost_customers].[GP], [Boost_customers].[GMV], [Boost_customers].[PM] 
FROM [Boost_customers]]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

: 

In [ ]:
output['output']

'1,892,014,583.7895546'

In [128]:
out = output['output']

In [129]:

def graph_code_generation(model_answer):
    prompt = """Begin!
    Task: 
    -import pandas and plotly
    -convert the following text {} to dataframe using python
    -create a python code to vizualize this dataframe
    -if you are using datetime handle NameError: name 'datetime' is not defined
    - dont write ```python ....```
    - if you are dealing with dates sort them asc
    output: should be python code only
    """.format(model_answer)

    # Create agent executor with prompt and tools
    agent_executor = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
                stop=None,
       
    )
 

    graph = agent_executor.choices[0].message.content
    
    return graph

graph = graph_code_generation(out)
print(graph)

import pandas as pd
import plotly.graph_objects as go

# Creating dataframe
data = {'Month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October'],
        'Sales': [896.725, 2285.181, 1009.616, 1615.767, 437.0, 1722.637, 1153.966, 836.757, 1735.652, 1218.499]}

df = pd.DataFrame(data)

# Sorting dataframe by month
df['Month'] = pd.to_datetime(df['Month'], format='%B', errors='coerce').dt.month
df = df.sort_values('Month')

# Visualizing the dataframe
fig = go.Figure(data=[go.Bar(x=df['Month'], y=df['Sales'], text=df['Sales'], textposition='outside')])
fig.update_layout(title_text='Total Sales per Month in 2023', xaxis_title='Month', yaxis_title='Sales')
fig.show()


In [130]:
exec(graph)

In [19]:
openai.__version__

'0.28.0'

In [20]:
PROMPT = """ 
Given an input question, first create sql server query to run,  
then look at the results of the query second creat python code to run  a graph of the result then  return the answer.  
The question: {question}
"""
question = "what is top 5 products names in terms of amount in each country"

In [25]:
out=agent_executor.invoke(question)

print(out['output'])



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: BL_Mapping_Table, Boost_customers, Customers_DAX_table, DAX_table, GMV_Calender_DAX_table, GMV_DimDate_DAX_table, Same_period, dimDate, dimDateEX, dim_B2B_Sales_Channel, dim_Target, dim_allusers, dim_balance, dim_categories, dim_industry, dim_log_status, dim_parent_categories, dim_payment_methods, dim_pos_sales_channel, dim_products, dim_segments, dim_stores, dim_users_b2b, dim_users_b2c, dim_users_pos, exploration, fact_b2b_trans, fact_b2c_trans, fact_points, fact_pos_trans, fact_salla_trans, fact_transactions, fact_vouchers
Thought:I should query the schema of the most relevant tables, and please return the output as a string.
Action: sql_db_schema
Action Input: dim_products
Observation: 
CREATE TABLE dim_products (
	product_id INTEGER NOT NULL, 
	product_name VARCHAR(250) COLLATE Latin1_General_100_BIN2_UTF8 NULL, 
	category_id INTEGER NULL, 
	CONSTRAINT [PK_product_id] PRIMARY KEY (prod

In [1]:
print(out['output'])

NameError: name 'out' is not defined

In [18]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [ ]:
#--

In [19]:
print(out['output'])

I don't know


In [20]:
 AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

NameError: name 'agent' is not defined